In [1]:
import pandas as pd
import pyarrow
import fastparquet
from geopy.geocoders import Nominatim



In [2]:
# df = pd.read_csv('data/uber_data.csv')

df = pd.read_parquet('data/yellow_tripdata_2024-01.parquet')


FileNotFoundError: [Errno 2] No such file or directory: 'data/yellow_tripdata_2024-01.parquet'

In [ ]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [ ]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0,1
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0,2
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0,3
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN,2964619
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN,2964620
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN,2964621
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN,2964622


In [ ]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

In [ ]:
datetime_dim['datetime_id'] = datetime_dim.index

In [ ]:
datetime_dim[[
    'datetime_id',
   'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'pick_hour',
    'pick_day',
    'pick_month',
    'pick_year',
    'pick_weekday',
    'drop_hour',
    'drop_day',
    'drop_month',
    'drop_year',
    'drop_weekday'
]]


,datetime_id,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2024-01-01 00:57:55,2024-01-01 01:17:43,0,1,1,2024,0,1,1,1,2024,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,0,1,1,2024,0,0,1,1,2024,0
2,2,2024-01-01 00:17:06,2024-01-01 00:35:01,0,1,1,2024,0,0,1,1,2024,0
3,3,2024-01-01 00:36:38,2024-01-01 00:44:56,0,1,1,2024,0,0,1,1,2024,0
4,4,2024-01-01 00:46:51,2024-01-01 00:52:57,0,1,1,2024,0,0,1,1,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2964619,2024-01-31 23:45:59,2024-01-31 23:54:36,23,31,1,2024,2,23,31,1,2024,2
2964620,2964620,2024-01-31 23:13:07,2024-01-31 23:27:52,23,31,1,2024,2,23,31,1,2024,2
2964621,2964621,2024-01-31 23:19:00,2024-01-31 23:38:00,23,31,1,2024,2,23,31,1,2024,2
2964622,2964622,2024-01-31 23:07:23,2024-01-31 23:25:14,23,31,1,2024,2,23,31,1,2024,2


In [ ]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]


trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]


In [ ]:
rate_code_name = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride",
}
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_name)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,1.0,Standard rate
3,3,1.0,Standard rate
4,4,1.0,Standard rate


In [ ]:
lookup_location = pd.read_csv('https://raw.githubusercontent.com/DanielOX/Data-Engineering-Workflow-DUCKDB/main/nyc_taxi_trip_data/taxi%2B_zone_lookup.csv')[['LocationID','Zone']].drop_duplicates().reset_index(drop=True)
location_dim = lookup_location.rename(columns={'LocationID':'location_id','Zone':'location'})



In [ ]:
pickup_location_dim = df[['PULocationID']].reset_index(drop=True)
pickup_location_dim = pickup_location_dim.merge(location_dim, left_on="PULocationID", right_on="location_id")
pickup_location_dim['pickup_location_id']= pickup_location_dim.index
pickup_location_dim = pickup_location_dim.drop(columns=['PULocationID','location_id'])
pickup_location_dim = pickup_location_dim.rename(columns={'location':'pickup_location'})

dropoff_location_dim = df[['DOLocationID']].reset_index(drop=True)
dropoff_location_dim = dropoff_location_dim.merge(location_dim, left_on="DOLocationID", right_on="location_id")
dropoff_location_dim['dropoff_location_id']= dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim.drop(columns=['DOLocationID','location_id'])
dropoff_location_dim = dropoff_location_dim.rename(columns={'location':'dropoff_location'})

# pickup_location_dim = df[['pickup_latitude','pickup_longitude']].reset_index(drop=True)
# pickup_location_dim['pickup_location_id']= pickup_location_dim.index
# pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']]

# dropoff_location_dim = df[['dropoff_latitude','dropoff_longitude']].reset_index(drop=True)
# dropoff_location_dim['dropoff_location_id']= dropoff_location_dim.index
# dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]
# pickup_location_dim

In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_name'] = df['payment_type'].map(payment_type_name)
payment_type_dim['payment_type_id'] = payment_type_dim.index

payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

payment_type_dim.head(5)

,payment_type_id,payment_type,payment_type_name
0,0,2,Cash
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card


In [ ]:
fact_table = df.merge(passenger_count_dim,left_on = "trip_id", right_on ="passenger_count_id") \
                .merge(trip_distance_dim,left_on = "trip_id", right_on="trip_distance_id") \
                .merge(rate_code_dim,left_on = "trip_id", right_on='rate_code_id') \
                .merge(payment_type_dim,left_on = "trip_id", right_on='payment_type_id') \
                .merge(datetime_dim,left_on = "trip_id",right_on = 'datetime_id') \
                .merge(pickup_location_dim,left_on = "trip_id",right_on = 'pickup_location_id') \
                .merge(dropoff_location_dim,left_on = "trip_id", right_on = 'dropoff_location_id') \
                [['trip_id','VendorID','datetime_id','passenger_count_id','trip_distance_id','pickup_location_id',
                  'dropoff_location_id','rate_code_id','payment_type_id','fare_amount','extra','mta_tax',
                  'tip_amount','tolls_amount','improvement_surcharge','total_amount']]
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,pickup_location_id,dropoff_location_id,rate_code_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,0,0,0,17.70,1.00,0.5,0.00,0.00,1.0,22.70
1,1,1,1,1,1,1,1,1,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75
2,2,1,2,2,2,2,2,2,2,23.30,3.50,0.5,3.00,0.00,1.0,31.30
3,3,1,3,3,3,3,3,3,3,10.00,3.50,0.5,2.00,0.00,1.0,17.00
4,4,1,4,4,4,4,4,4,4,7.90,3.50,0.5,3.20,0.00,1.0,16.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2964619,2,2964619,2964619,2964619,2964619,2964619,2964619,2964619,15.77,0.00,0.5,2.00,0.00,1.0,21.77
2964620,2964620,1,2964620,2964620,2964620,2964620,2964620,2964620,2964620,18.40,1.00,0.5,2.34,0.00,1.0,25.74
2964621,2964621,2,2964621,2964621,2964621,2964621,2964621,2964621,2964621,19.97,0.00,0.5,0.00,0.00,1.0,23.97
2964622,2964622,2,2964622,2964622,2964622,2964622,2964622,2964622,2964622,23.88,0.00,0.5,5.58,0.00,1.0,33.46
